In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#载入数据集
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
#每个批次的大小,神经网络一次性放入的数据，100张图片
batch_size=10
#计算一共有多少个批次
n_batch=mnist.train.num_examples//batch_size#//代表整除
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    #初始化随机的变量，不知道为啥，别人都这么写，应该是卷积神经网络用的吧，后面的stddev不知道是啥，后续补充
    return tf.Variable(initial)
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    #初始定义为常量正值
    return tf.Variable(initial)
def conv2d(x,W):
    #strides为步长，第一个和最后一个位置必须为1.第二个代表x跨度，第三个代表y跨度[1,x_movement,y_movement,1]
    #padding. SAME与原图片一样， VALID抽取出来的比原来小
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def max_pool_2x2(x):
    #pooling，一种方法
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
xs=tf.placeholder(tf.float32,[None,784])
ys=tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)
x_image=tf.reshape(xs,[-1,28,28,1])#-1代表忽略xs的维度，也就是前面的None。1代表channel，灰白。
#x_image.shape.  [n_samples,28,28,1]

#conv1 layer
W_conv1=weight_variable([5,5,1,32])#patch 5x5,in_size=1,out_size=32.patch小方块的大小。in_size和out_size是图片厚度
b_conv1=bias_variable([32])

h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)#relu非线性化,output size= 28x28x32
h_pool1=max_pool_2x2(h_conv1)#output size=14x14x32

#conv2 layer
W_conv2=weight_variable([5,5,32,64])#patch 5x5,in_size=32,out_size=64.patch小方块的大小。in_size和out_size是图片厚度
b_conv2=bias_variable([64])

h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)#relu非线性化,output size= 14x14x64
h_pool2=max_pool_2x2(h_conv2)#output size=7x7x64

#func1 layer1
W_f1=weight_variable([7*7*64,1024])
b_f1=bias_variable([1024])

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])#[n_sample,7,7,64]->>[n_sample,7*7*64]
h_f1=tf.nn.relu(tf.matmul(h_pool2_flat,W_f1)+b_f1)
h_f1_drop=tf.nn.dropout(h_f1,keep_prob)#不知道

#func1 layer2
W_f2=weight_variable([1024,10])
b_f2=bias_variable([10])

prediction=tf.nn.softmax(tf.matmul(h_f1_drop,W_f2)+b_f2)


#the error between prediction and real data
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ys,logits=prediction))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction=tf.equal(tf.argmax(ys,1),tf.argmax(prediction,1))#argmax返回一维张量中最大值所在的位置
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))#cast将布尔转化为浮点
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(21):#这个循环代表所有图片训练21次
        for batch in range(n_batch):#这个循环代表把所有图片训练一次
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys})
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Iter'+str(_)+',Testing Accuracy'+str(acc))
        


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_14' with dtype float
	 [[Node: Placeholder_14 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_14', defined at:
  File "/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/anaconda3/lib/python3.6/site